In [1]:
import pandas as pd

from pathlib import Path
import csv

import os
import requests
import json

from dotenv import load_dotenv
load_dotenv()

True

In [2]:
## IEX Constants
iex_api_key = os.getenv("IEX_API_KEY")
iex_test_api_key = os.getenv("IEX_TEST_API_KEY")

## Redundant Assignment but improves Readability throughout code 
real_token = iex_api_key
test_token = iex_test_api_key

base_url_iex = 'https://cloud.iexapis.com/stable/'
sandbox_url = 'https://sandbox.iexapis.com/stable/'

## IEX Status Test 
test_resp = requests.get(base_url_iex + 'status')
test_resp

<Response [200]>

In [3]:
token_status = test_token ## Set to either real token or test token for IEX

In [4]:
## Load IEX to get ETF statistics 
def get_IEX_statistics(stock_ticker, token=token_status):
    if token == test_token:
        resp_data = requests.get(sandbox_url+'stock/'+stock_ticker+'/stats/?token='+test_token)
        data_json = resp_data.json()
    elif token == real_token:
        resp_data = requests.get(base_url_iex+'stock/'+stock_ticker+'/stats/?token='+real_token)
        data_json = resp_data.json()
        
    return data_json

In [5]:
## FMP Constants 
fmpbase_urlv3 = 'https://fmpcloud.io/api/v3/'
fmpbase_urlv4 = 'https://fmpcloud.io/api/v4/'
api_key = os.getenv("FMP_CLOUD_API_KEY")

## FMP Functions 
# def get_FMP_historical_data(symbol, startDate=start_date, endDate=end_date, apiKey=api_key):
#     url_hist_price = fmpbase_urlv3+'historical-price-full/'
#     url_hist_query_with_date = url_hist_price+symbol+'?from='+startDate+'&to='+endDate+'&apikey='+apiKey
#     resp_data = requests.get(url_hist_query_with_date)
#     json_ = resp_data.json()
#     data = json_['historical']
#     df = pd.DataFrame(data)
#     df.rename(columns={'date':'Date'},inplace=True)
#     df['Date'] = pd.to_datetime(df['Date'])
#     df = df.reindex(index=df.index[::-1]) ## Reverse the DataFrame 
#     df.set_index('Date',inplace=True)
#     df.drop(columns='label',inplace=True)
#     return df

def get_float_data_FMP(symbol):
    url_float_shares = fmpbase_urlv4+'shares_float?symbol='
    url_query_float_data = url_float_shares+symbol+'&apikey='+api_key
    resp_data = requests.get(url_query_float_data)
    #df = pd.DataFrame(resp_data.json())
    json_ = resp_data.json()
    return json_[0]

def get_company_profile_FMP_json(symbol):
    ## https://fmpcloud.io/api/v3/profile/AAPL?apikey='yourkeyhere'
    url_company_profile_url = fmpbase_urlv3+'profile/'+symbol+'?apikey='+api_key
    resp_data = requests.get(url_company_profile_url)
    json_response = resp_data.json()
    return json_response[0]

def save_and_export_raw_df_csv(data, symbol):
    path = ('../FilesExportIndividualStockDFs/'+symbol+'_combined_df.csv')
    data.to_csv(path)

In [6]:
## Use pickle module to import and export and save files
import pickle
def load_obj(path):
    with open(path, 'rb') as f:
        return pickle.load(f)
def save_obj(obj, path ):
    with open(path, 'wb') as f:
        pickle.dump(obj, f, pickle.HIGHEST_PROTOCOL)

In [7]:
## Import files using Pickle
imported_data_dict = {}
imported_key_list = []
## Need to set length manually - check files for max length based off file name 
def import_data(loop_length):
    x=0
    y=49
    increment=50
    
    while x < loop_length:
        str_symbol1 = str(x)
        str_symbol2 = str(y)

        pkl_path = Path('../FilesExportCompleteFMP_big/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
        data_obj = load_obj(pkl_path)
        data_obj_key_list = []
        for key in data_obj.keys():
            data_obj_key_list.append(key)  
        for symbol in data_obj_key_list:
            data = data_obj[symbol]
            imported_data_dict[symbol] = data

        x += increment
        y += increment
        if y > loop_length: y = loop_length
        
    for key in imported_data_dict.keys():
        imported_key_list.append(key) 

In [8]:
## Need to create data structure for rest of data 

In [9]:
import_data(loop_length=199)

In [10]:
len(imported_key_list)

33

In [11]:
## Sort index anytime accessing data 
test_df = imported_data_dict['AAPL'].sort_index()
test_df.iloc[-1]['close']

149.800003

In [12]:
full_length_ = len(imported_data_dict['AAPL'])
full_length_ = 1469

In [13]:
## Test get get_IEX_statistics
appl_iex_stats = get_IEX_statistics(stock_ticker='AAPL')
appl_iex_stats

{'companyName': 'Apple Inc',
 'marketcap': 2576629785388,
 'week52high': 165.9,
 'week52low': 119.89,
 'week52highSplitAdjustOnly': 166.7,
 'week52lowSplitAdjustOnly': 121.18,
 'week52change': 0.3671282168562986,
 'sharesOutstanding': 16596259231,
 'float': 0,
 'avg10Volume': 90444082,
 'avg30Volume': 77028469,
 'day200MovingAvg': 147.17,
 'day50MovingAvg': 151.47,
 'employees': 148372,
 'ttmEPS': 11.78,
 'ttmDividendRate': 0.8924497582902171,
 'dividendYield': 0.00551016813341884,
 'nextDividendDate': '',
 'exDividendDate': '2021-10-25',
 'nextEarningsDate': '2022-01-19',
 'peRatio': 13.864157406929397,
 'beta': 1.4433275385120816,
 'maxChangePercent': 59.78444084136919,
 'year5ChangePercent': 5.038039755745583,
 'year2ChangePercent': 1.39890547166229,
 'year1ChangePercent': 0.356405480894378,
 'ytdChangePercent': 0.1964753793008941,
 'month6ChangePercent': 0.2659979499491978,
 'month3ChangePercent': 0.05798938665007065,
 'month1ChangePercent': 0.03034343473588546,
 'day30ChangePercen

In [14]:
appl_fmp_profile = get_company_profile_FMP_json('AAPL')
appl_fmp_profile

{'symbol': 'AAPL',
 'price': 160.24,
 'beta': 1.205714,
 'volAvg': 78850734,
 'mktCap': 2628961697792,
 'lastDiv': 0.865,
 'range': '116.21-165.7',
 'changes': 3.430008,
 'companyName': 'Apple Inc.',
 'currency': 'USD',
 'cik': '0000320193',
 'isin': 'US0378331005',
 'cusip': '037833100',
 'exchange': 'Nasdaq Global Select',
 'exchangeShortName': 'NASDAQ',
 'industry': 'Consumer Electronics',
 'website': 'http://www.apple.com',
 'description': "Apple Inc. (Apple) designs, manufactures and markets mobile communication and media devices, personal computers, and portable digital music players, and a variety of related software, services, peripherals, networking solutions, and third-party digital content and applications. The Company's products and services include iPhone, iPad, Mac, iPod, Apple TV, a portfolio of consumer and professional software applications, the iOS and OS X operating systems, iCloud, and a variety of accessory, service and support offerings. The Company also delivers 

In [15]:
appl_fmp_profile['mktCap']

2628961697792

In [16]:
aapl_float_data = get_float_data_FMP('AAPL')
aapl_float_data

{'symbol': 'AAPL',
 'date': '2021-11-29',
 'freeFloat': 99.89598173273845,
 'floatShares': 16389334347.0,
 'outstandingShares': 16406400000.0,
 'source': 'https://www.sec.gov/Archives/edgar/data/320193/000032019321000105/aapl-20210925.htm'}

In [17]:
aapl_float_data['floatShares']

16389334347.0

In [18]:
# data = {
# 	'Fundamentals':                    			##
# 	{
# 		 'sharesOutstanding': 17213236419,		## Sourced from IEX for ETFs, FMP for Equity
# 		 'floatShares': 16389334347.0,			## Can't source for ETFs at the moment - can get from FMP for Equity
# 		 'debt_ratio': 'debt_ratioValue',		## Blank - no source yet 
# 		 'exchange': 'Nasdaq Global Select',    ## Sourced from FMP 
# 		 'final_close_price': 149.800003,		## Sourced from iex_statistics
# 		 'peRatio': 14.407047826172997,			## Sourced from iex_statistics, =0 for ETFs
# 		 'beta': 1.4575864187912306,			## Sourced from iex_statistics for Equity, =0 for ETFs
# 		 'week52high': 169.1,					## Sourced from iex_statistics
# 		 'week52low': 118.18,					## Sourced from iex_statistics
# 		 'week52change': 0.4124429632647193,    ## Sourced from iex_statistics
# 		 'avg10Volume': 102534515,				## Sourced from iex_statistics
# 		 'avg30Volume': 79254408,				## Sourced from iex_statistics
# 		 'marketcap_IEX': 2663047538957,		## Sourced from iex_statistics
# 		 'marketcap_FMP': 2656852508672			## Sourced from FMP	
# 	},
# 	'dataFrame':time_series_df,							## FMP historical merged with Nasdaq Short Data and SEC FTD Data
# 	'companyProfile':FMP_company_profile_json,         	## Sourced from FMP, otherwise =0
# 	'floatData':FMP_float_data_df,   					## Sourced from FMP, otherwise =0
# 	'textNews':['article1','article2','article3'],    	## Not sourced
# 	'returns':'returns_data',    						## Not calculated
# 	'iex_statistics':'iex_statistics_data_json'     	## Added to completed data_dicts, both ETFs and Equity
# }

In [19]:
data_dict_all = {}
data_dict_full = {}
data_dict_uneven={}

symbol_list_all = []
symbol_list_full = []
symbol_list_uneven=[]

total_length = 33915 ## Number of possible symbols 
full_length_ = 1469 ## Length of a full data fram (2016-01 to 2021-10)

In [25]:
loop_length = 199
x=0
y=49
increment=50



iex_token = test_token ## IEX Token Status 

while x < loop_length:
    str_symbol1 = str(x)
    str_symbol2 = str(y)

    ## Load time series data - Historical(FMP), FTD(SEC), and Short Interest (FINRA)
    pkl_path = Path('../FilesExportCompleteFMP_big/data_complete_'+str_symbol1+'_'+str_symbol2+'.pkl')
    data_obj = load_obj(pkl_path)
    data_obj_key_list = []
    for key in data_obj.keys():
        data_obj_key_list.append(key)  
    for symbol in data_obj_key_list:
        err_symbol = symbol
        ## Get time series data 
        time_series_df = data_obj[symbol].sort_index() ## Create dataframe from pkl object
        try:
            iex_stats = get_IEX_statistics(stock_ticker=symbol,token=iex_token)        
            sharesOutstanding = iex_stats['sharesOutstanding']
            peRatio = iex_stats['peRatio']
            beta = iex_stats['beta']
            week52high = iex_stats['week52high']
            week52low = iex_stats['week52low']
            week52change = iex_stats['week52change']
            avg10Volume = iex_stats['avg10Volume']
            avg30Volume = iex_stats['avg30Volume']
            marketcap_IEX = iex_stats['marketcap']
        except:
            sharesOutstanding = 0
            peRatio = 0
            beta = 0
            week52high = 0
            week52low = 0
            week52change = 0
            avg10Volume = 0
            avg30Volume = 0
            marketcap_IEX = 0
        
        ## FMP 
        try:
            fmp_profile = get_company_profile_FMP_json(symbol)
            exchange = fmp_profile['exchangeShortName']
            marketcap_FMP = fmp_profile['mktCap']
        except:
            fmp_profile = 0
            exchange = 0
            marketcap_FMP = 0
        try:
            float_data = get_float_data_FMP(symbol)
            floatShares = float_data['floatShares']
        except:
            float_data = 0
            floatShares = 0
        
        
        ## Create data_dict: 
        data = {
        'Fundamentals':                    			##
        {
            'sharesOutstanding': sharesOutstanding,		## Sourced from IEX for ETFs, FMP for Equity
             'floatShares': floatShares,			## Can't source for ETFs at the moment - can get from FMP for Equity
             #'debt_ratio': 'debt_ratioValue',		## Blank - no source yet 
             'exchange': exchange,    ## Sourced from FMP 
             'final_close_price': time_series_df.iloc[-1]['close'],		## Sourced from iex_statistics
             'peRatio': peRatio,			## Sourced from iex_statistics, =0 for ETFs
             'beta': beta,			## Sourced from iex_statistics for Equity, =0 for ETFs
             'week52high': week52high,					## Sourced from iex_statistics
             'week52low': week52low,					## Sourced from iex_statistics
             'week52change': week52change,    ## Sourced from iex_statistics
             'avg10Volume': avg10Volume,				## Sourced from iex_statistics
             'avg30Volume': avg30Volume,				## Sourced from iex_statistics
             'marketcap_IEX': marketcap_IEX,		## Sourced from iex_statistics
             'marketcap_FMP': marketcap_FMP			## Sourced from FMP	
        },
        'dataFrame':time_series_df,							## FMP historical merged with Nasdaq Short Data and SEC FTD Data
        'companyProfile':fmp_profile,         	## Sourced from FMP, otherwise =0
        'floatData':float_data,   					## Sourced from FMP, otherwise =0
        'textNews':['article1','article2','article3'],    	## Not sourced
        'returns':'returns_data',    						## Not calculated
        'iex_statistics': iex_stats     	## Added to completed data_dicts, both ETFs and Equity
        }
        ## Export individual data_dict 
        export_path = Path('../FilesExport_IndividualDataDict/'+symbol+'data_dict.pkl')
        save_obj(data,export_path)
        
                
        if len(time_series_df) == full_length_:
            data_dict_full[symbol] = data
            symbol_list_full.append(symbol)
        else:
            data_dict_uneven[symbol] = data
            symbol_list_uneven.append(symbol)
        
        ## Append all data 
        data_dict_all[symbol] = data
        ## Append symbol to list 
        symbol_list_all.append(symbol)  

    x += increment
    y += increment
    if y > loop_length: y = loop_length

## If while loop finishes - export all data 
pkl_path = Path('../Resources/data_dict_all_test.pkl')
save_obj(data_dict_all,pkl_path)
pkl_path = Path('../Resources/data_dict_full_test.pkl')
save_obj(data_dict_full,pkl_path)
pkl_path = Path('../Resources/data_dict_uneven_test.pkl')
save_obj(data_dict_uneven,pkl_path)


symbol_list_alldf = pd.DataFrame(symbol_list_all)
symbol_list_fulldf = pd.DataFrame(symbol_list_full)
symbol_list_unevendf = pd.DataFrame(symbol_list_uneven)

## Export list
list_path = ('../Resources/all_symbol_list_test.csv')
symbol_list_alldf.to_csv(list_path)
list_path = ('../Resources/symbol_list_full_test.csv')
symbol_list_fulldf.to_csv(list_path)
list_path = ('../Resources/symbol_list_uneven_test.csv')
symbol_list_unevendf.to_csv(list_path)

In [26]:
err_symbol

'ABB'

In [27]:
err_data = get_float_data_FMP(err_symbol)
err_data

{'symbol': 'ABB',
 'date': '2021-11-29',
 'freeFloat': 81.2342711986921,
 'floatShares': 1618998973.0,
 'outstandingShares': 1992999936.0,
 'source': ''}

In [29]:
data_dict_all['AAPL']

{'Fundamentals': {'sharesOutstanding': 16794397925,
  'floatShares': 16389334347.0,
  'exchange': 'NASDAQ',
  'final_close_price': 149.800003,
  'peRatio': 13.599786461730238,
  'beta': 1.4861429740238898,
  'week52high': 173.9,
  'week52low': 116.72,
  'week52change': 0.3605139156671686,
  'avg10Volume': 93362050,
  'avg30Volume': 78268329,
  'marketcap_IEX': 2650999711968,
  'marketcap_FMP': 2628961697792},
 'dataFrame':                   open        high         low       close    adjClose  \
 Date                                                                     
 2016-01-04   25.652500   26.342501   25.500000   26.337500   24.251429   
 2016-01-05   26.437500   26.462500   25.602501   25.677500   23.643715   
 2016-01-06   25.139999   25.592501   24.967501   25.174999   23.181011   
 2016-01-07   24.670000   25.032499   24.107500   24.112499   22.202663   
 2016-01-08   24.637501   24.777500   24.190001   24.240000   22.320068   
 ...                ...         ...         ...  